In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
tf.random.set_seed(7)

In [11]:
train_dir = '../dataset/train/'
train_df = pd.read_csv('../dataset/train.csv')

test_dir = '../dataset/test/'

In [13]:
train_df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [14]:
train_df.shape

(6469, 2)

In [15]:
train_df.Class.unique()

array(['Miscellaneous', 'Candle', 'Snowman', 'Airplane', 'Christmas_Tree',
       'Jacket'], dtype=object)

In [16]:
train_df.Class.value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [ ]:
label_map = {
    'Miscellaneous': 0,
    'Christmas_tree': 1,
    
}

In [53]:
def add_path(img):
    return '../dataset/train/'+img

In [54]:
filenames = train_df.Image.apply(add_path)
labels = train_df.Class

dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))

In [55]:
print(next(iter(dataset)))

(<tf.Tensor: shape=(), dtype=string, numpy=b'../dataset/train/image3476.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'Miscellaneous'>)


In [56]:
dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None))

In [57]:
def parse_function(filename, label):
    image_str = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_str,channels=3)
    image = tf.image.convert_image_dtype(image,tf.float32)
    return image,label

In [58]:
def train_preprocess(image,label):
    image = tf.image.random_flip_left_right(image)
    image = tf.clip_by_value(image,0.0,1.0)
    label = 
    return image,label

In [69]:
def process_label(label):
    

In [71]:
process_label(labels[1])

[0 0 0 0 0 0]


In [72]:
labels[0]

'Miscellaneous'

In [59]:
BATCH_SIZE = 128

In [60]:
dataset = dataset.shuffle(len(filenames))
dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset = dataset.map(train_preprocess, num_parallel_calls=4)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(1)

In [61]:
validation_data = dataset.take(int(train_df.shape[0]*0.2))
training_data = dataset.skip(int(train_df.shape[0]*0.2))

In [66]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224


def create_model():
    IMAGE_SHAPE = (IMAGE_HEIGHT,IMAGE_WIDTH,3)
    base_model = tf.keras.applications.ResNet50(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')
    
    pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
    output_layer = tf.keras.layers.Dense(1)
    
    model = tf.keras.models.Sequential([
    base_model,
    pooling_layer,
    output_layer
    ])
    lr = 1e-5
    optimizer = tf.keras.optimizers.RMSprop(lr=lr)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = ['accuracy']

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

In [63]:
model = create_model()

In [64]:
checkpoint_filepath = './checkpoints/training_resnet'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',verbose=1,
    save_best_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [65]:
history = model.fit(dataset,epochs=5,
                    verbose=1,validation_data=validation_data,
                    callbacks=[early_stopping])

Epoch 1/5


UnimplementedError:  Cast string to float is not supported
	 [[node Cast_2 (defined at <ipython-input-65-24f3edf2fb70>:3) ]] [Op:__inference_train_function_48733]

Function call stack:
train_function
